# Testing phase - Zoo.csv

In [46]:
import sys
sys.path.append('../src/')
from rdr import RDR

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Import Model Libraries
White box Model :
- Decision Tree Classifier

Black box Model :
- Random Forest
- Support Vector Machine
- XGBoost
- MLP
- TabNet

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier

## 1. Load dataset

In [49]:
df = pd.read_csv('../data/zoo.csv')
display(df.head())
display(df.isnull().sum())

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


animal_name    0
hair           0
feathers       0
eggs           0
milk           0
airborne       0
aquatic        0
predator       0
toothed        0
backbone       0
breathes       0
venomous       0
fins           0
legs           0
tail           0
domestic       0
catsize        0
class_type     0
dtype: int64

In [50]:
label = df['animal_name']
features = df.drop(['animal_name'], axis=1)

le = LabelEncoder()
le.fit(label)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

## 2. Comparison between conventional models with RDR model

### 2.0. Create RDR base model

In [51]:
rdr = RDR(
        antecedent="human",
        categorical_attr=features.columns.to_list(),
        total_precedent=3)

### 2.1. Decision Tree vs RDR

In [52]:
# CREATE AND TRAIN DECISION TREE CLASSIFIER
dtf = DecisionTreeClassifier()
dtf.fit(X_train, y_train)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = dtf.predict(features)

# CREATE AND TRAIN RDR MODEL
dtf_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dtf_pred = dtf.predict(X_test)
dtf_rdr_pred = dtf_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,dtf_pred))
print(metrics.accuracy_score(y_test, dtf_rdr_pred))
print(metrics.accuracy_score(dtf_pred, dtf_rdr_pred))

0.0
0.0
1.0


### 2.2. Random Forest vs RDR

In [53]:
# CREATE AND TRAIN RANDOM FOREST CLASSIFIER
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = rf.predict(features)

# CREATE AND TRAIN RDR MODEL
rf_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
rf_pred = rf.predict(X_test)
rf_rdr_pred = rf_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,rf_pred))
print(metrics.accuracy_score(y_test, rf_rdr_pred))
print(metrics.accuracy_score(rf_pred, rf_rdr_pred))

0.0
0.0
1.0


### 2.3. SVM vs RDR

In [54]:
# CREATE AND TRAIN SUPPORT VECTOR MACHINE
svm = SVC()
svm.fit(X_train, y_train)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = svm.predict(features)

# CREATE AND TRAIN RDR MODEL
svm_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
svm_pred = svm.predict(X_test)
svm_rdr_pred = svm_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,svm_pred))
print(metrics.accuracy_score(y_test, svm_rdr_pred))
print(metrics.accuracy_score(svm_pred, svm_rdr_pred))

0.0
0.0
1.0


### 2.4. XGBoost vs RDR

In [55]:
# CREATE AND TRAIN XGBOOST CLASSIFIER
train_dmatrix = xgb.DMatrix(data=X_train, label=le.transform(y_train))
xgboost = xgb.train({}, train_dmatrix)

dmatrix = xgb.DMatrix(data=features, label=le.transform(label))
xgb_pred = xgboost.predict(dmatrix)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = le.inverse_transform(np.round(xgb_pred).astype(int))

# CREATE AND TRAIN RDR MODEL
xgb_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dmatrix = xgb.DMatrix(data=X_test, label=le.transform(y_test))
xgb_pred = xgboost.predict(dmatrix)
xgb_pred = le.inverse_transform(np.round(xgb_pred).astype(int))

xgb_rdr_pred = xgb_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,xgb_pred))
print(metrics.accuracy_score(y_test, xgb_rdr_pred))
print(metrics.accuracy_score(xgb_pred, xgb_rdr_pred))

0.047619047619047616
0.047619047619047616
1.0


### 2.5. MLP vs RDR

In [56]:
# CREATE AND TRAIN MLP CLASSIFIER
mlp = MLPClassifier(max_iter=100, batch_size=5, hidden_layer_sizes=(10, 15, 30), learning_rate='constant', learning_rate_init=0.1)
mlp.fit(X_train, y_train)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = mlp.predict(features)

In [57]:
# CREATE AND TRAIN RDR MODEL
mlp_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - MLP AND RDR
mlp_pred = mlp.predict(X_test)
mlp_rdr_pred = mlp_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,mlp_pred))
print(metrics.accuracy_score(y_test, mlp_rdr_pred))
print(metrics.accuracy_score(mlp_pred, mlp_rdr_pred))

0.0
0.0
1.0


### 2.6. TabNet vs RDR

In [58]:
# CREATE AND TRAIN TabNet Classifier
tabnet = TabNetClassifier()
tabnet.fit(X_train.values, y_train, max_epochs=100)

# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = tabnet.predict(features.values)

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 74 | loss: 0.0     |  0:00:00s
epoch 75 | loss: 0.0     |  0:00:00s
epoch 76 | loss: 0.0     |  0:00:00s
epoch 77 | loss: 0.0     |  0:00:00s
epoch 78 | loss: 0.0     |  0:00:00s
epoch 79 | loss: 0.0     |  0:00:00s
epoch 80 | loss: 0.0     |  0:00:00s
epoch 81 | loss: 0.0     |  0:00:00s
epoch 82 | loss: 0.0     |  0:00:00s
epoch 83 | loss: 0.0     |  0:00:00s
epoch 84 | loss: 0.0     |  0:00:00s
epoch 85 | loss: 0.0     |  0:00:00s
epoch 86 | loss: 0.0     |  0:00:00s
epoch 87 | loss: 0.0     |  0:00:00s
epoch 88 | loss: 0.0     |  0:00:00s
epoch 89 | loss: 0.0     |  0:00:00s
epoch 90 | loss: 0.0     |  0:00:00s
epoch 91 | loss: 0.0     |  0:00:00s
epoch 92 | loss: 0.0     |  0:00:00s
epoch 93 | loss: 0.0     |  0:00:00s
epoch 94 | loss: 0.0     |  0:00:00s
epoch 95 | loss: 0.0     |  0:00:00s
epoch 96 | loss: 0.0     |  0:00:00s
epoch 97 | loss: 0.0     |  0:00:00s
epoch 98 | loss: 0.0     |  0:00:00s
epoch 99 | loss: 0.0     |  0:00:00s


In [59]:
# CREATE AND TRAIN RDR MODEL
tabnet_rdr = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - TabNet AND RDR
tabnet_pred = tabnet.predict(X_test.values)
tabnet_rdr_pred = tabnet_rdr.predict(X_test)

print(metrics.accuracy_score(y_test,tabnet_pred))
print(metrics.accuracy_score(y_test, tabnet_rdr_pred))
print(metrics.accuracy_score(tabnet_pred, tabnet_rdr_pred))

0.0
0.0
1.0


## 5. Accuracy Comparison between RDR model and conventional models

In [60]:
dtf_accuracy = metrics.accuracy_score(y_test,dtf_pred)
rf_accuracy = metrics.accuracy_score(y_test,rf_pred)
xgb_accuracy = metrics.accuracy_score(y_test,xgb_pred)
svm_accuracy = metrics.accuracy_score(y_test,svm_pred)
mlp_accuracy = metrics.accuracy_score(y_test,mlp_pred)
tabnet_accuracy = metrics.accuracy_score(y_test,tabnet_pred)

dtf_rdr_accuracy = metrics.accuracy_score(y_test, dtf_rdr_pred)
rf_rdr_accuracy = metrics.accuracy_score(y_test, rf_rdr_pred)
xgb_rdr_accuracy = metrics.accuracy_score(y_test, xgb_rdr_pred)
svm_rdr_accuracy = metrics.accuracy_score(y_test, svm_rdr_pred)
mlp_rdr_accuracy = metrics.accuracy_score(y_test, mlp_rdr_pred)
tabnet_rdr_accuracy = metrics.accuracy_score(y_test, tabnet_rdr_pred)

accuracy_table = []
accuracy_table.append([
    "Decision Tree",
    dtf_accuracy,
    dtf_rdr_accuracy,
    metrics.accuracy_score(dtf_pred, dtf_rdr_pred)
])

accuracy_table.append([
    "Random Forest",
    rf_accuracy,
    rf_rdr_accuracy,
    metrics.accuracy_score(rf_pred, rf_rdr_pred)
])

accuracy_table.append([
    "XGBoost",
    xgb_accuracy,
    xgb_rdr_accuracy,
    metrics.accuracy_score(xgb_pred, xgb_rdr_pred)
])

accuracy_table.append([
    "Support Vector Machine",
    svm_accuracy,
    svm_rdr_accuracy,
    metrics.accuracy_score(svm_pred, svm_rdr_pred)
])

accuracy_table.append([
    "Multi-layer Perceptron",
    mlp_accuracy,
    mlp_rdr_accuracy,
    metrics.accuracy_score(mlp_pred, mlp_rdr_pred)
])

accuracy_table.append([
    "TabNet",
    tabnet_accuracy,
    tabnet_rdr_accuracy,
    metrics.accuracy_score(tabnet_pred, tabnet_rdr_pred)
])

accuracy_table = pd.DataFrame(accuracy_table, columns=["Model", "Accuracy", "RDR Accuracy", "Difference"])
pd.set_option('display.precision', 15)

display(accuracy_table)

,Model,Accuracy,RDR Accuracy,Difference
0,Decision Tree,0.000000000000000,0.000000000000000,1.0
1,Random Forest,0.000000000000000,0.000000000000000,1.0
2,XGBoost,0.047619047619048,0.047619047619048,1.0
3,Support Vector Machine,0.000000000000000,0.000000000000000,1.0
4,Multi-layer Perceptron,0.000000000000000,0.000000000000000,1.0
5,TabNet,0.000000000000000,0.000000000000000,1.0


In [61]:
from graphviz import Digraph
from node import Node
from IPython.display import display


In [62]:
def visualize_tree(node: Node, dot=None):
    if dot is None:
        dot = Digraph()
        dot.node('root', str(node))
    
    if node.getExceptNode():
        dot.node(str(id(node.getExceptNode())), str(node.getExceptNode()))
        dot.edge('root', str(id(node.getExceptNode())), label='except')
        visualize_tree(node.getExceptNode(), dot)
    
    if node.getElseNode():
        dot.node(str(id(node.getElseNode())), str(node.getElseNode()))
        dot.edge('root', str(id(node.getElseNode())), label='else')
        visualize_tree(node.getElseNode(), dot)
    
    return dot

# Example usage:
dot = visualize_tree(dtf_rdr._root)
dot.attr(size='10,10')  # Set the size of the image (adjust as needed)
display(dot)  # Display the decision tree visualization directly in VSCode

## Find how to define the content for each node in the visualization